In [1]:
CUDA_LAUNCH_BLOCKING="1"

In [2]:
#!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"

In [3]:
#!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

In [4]:
!pip install indic-nlp-library

In [5]:
# from google.colab import drive
# drive.mount("/content/gdrive")
# import os
# import sys
# path = '/content/gdrive/My Drive/'
# os.chdir(path)
# print(os.getcwd())

In [6]:
!head train2.csv

head: cannot open 'train2.csv' for reading: No such file or directory


In [7]:
#loading libraries
import torch
torch.cuda.empty_cache()
import torch.nn as nn
from torch import optim
from torch.utils import data
import torch.nn.functional as F
print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import string
import re
import random
from nltk.tokenize import word_tokenize
from unicodedata import normalize
import numpy as np
from numpy import array

1.7.0
cuda


In [8]:
from indicnlp.tokenize import indic_tokenize 

In [9]:
# from indicnlp.normalize.indic_normalize import DevanagariNormalizer
# factory = DevanagariNormalizer()

from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
factory=IndicNormalizerFactory()
normalizer=factory.get_normalizer("hi",remove_nuktas=False)

Data Cleaning

In [10]:
#Lowercase, trim, and remove non-letter characters
def normalizeString(w):
  #insert space between words and punctuations
  w = re.sub(r"([?.!,¿-])", r" \1 ", w)
  w = re.sub(r"[().!,?-]+", r" ", w)
  w = re.sub(r'[" "]+', " ", w) #multiple spaces
  w = w.lower().strip() #lowercase #Remove white spaces
  return w

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
abbr = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"here's": "here is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"y'all":"you all",
"let's": "let us"
}

In [13]:
import csv
data = []
with open('/kaggle/input/traincsv/train.csv') as csvfile:
    csvReader = csv.reader(csvfile, delimiter=',')
    for row in csvReader:
        data.append([normalizeString(row[1]), normalizeString(row[2])])

In [14]:
for l in data:
  for w in l[1].split():
    if(w in abbr.keys()):
      #print(w)
      l[1] = l[1].replace(w, abbr[w])
      break

In [15]:
data = data[1:]

In [16]:
test = []
with open('/kaggle/input/testhindi/testhindistatements.csv') as csvfile:
    csvReader = csv.reader(csvfile, delimiter=',')
    for row in csvReader:
        test.append([row[2]])

In [17]:
#start of sentence
SOS_token = 0
#end of sentence
EOS_token = 1

#helper class - converting textual data to numeric form, creating vocabulary
class Lang:
  def __init__(self, name):
    self.name = name
    #word -> index
    self.word2index = {"SOS" : 0, "EOS" : 1, "UNK" : 2}
    #count of each word to later replace rare words
    self.word2count = {}
    #index -> word
    self.index2word = {0: "SOS", 1: "EOS", 2 : "UNK"}
    self.n_words = 3  # Count SOS, EOS, UNK tokens

  def addSentence(self, sentence):
    sentence_split = sentence.split(' ')
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      # word to index mapping
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

In [18]:
test = test[1:]

In [19]:
def clean_pairs(lines):
  cleaned = list()
  regex = re.compile('[%s]' % re.escape(string.punctuation))

  for pair in lines:
    clean_pair = list()
    
    #cleaning hindi phrases
    line = pair[0]
    line = normalizer.normalize(line)
    tokens = list()
    for t in indic_tokenize.trivial_tokenize(line): 
        tokens.append(t)
    line = tokens
    line = [word for word in line if not re.search(r'\d', word)]
    line = [regex.sub('', w) for w in line]
    #Replace the english characters
    line = [re.sub(r"[a-zA-Z]+?\s", "", w) for w in line]
    line = [re.sub(r'[-.।|,?;:<>&$₹]+','',w) for w in line]
    clean_pair.append(' '.join(line))

    #cleaning english phrases
    line = pair[1]
    #convert unicode line to plain ascii
    line = normalize('NFD', line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    line = word_tokenize(line) 
    line = [regex.sub('', w) for w in line]
    #replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    line = [re.sub(r"[^a-zA-Z0-9?.!,¿\-\/]+", " ", w) for w in line]
    line = [re.sub(r" - ", "-", w) for w in line]
    line = [word for word in line if word.isalpha()]
    clean_pair.append(' '.join(line))
    
    cleaned.append(clean_pair)
  return array(cleaned)

Filtering sentences that are too long

In [20]:
MAX_LENGTH = 60

def filterPair(p):
    return len(p[0].split(' ')) <= MAX_LENGTH and len(p[1].split(' ')) <= MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [21]:
def prepareData(lang1, lang2):
    #make Lang instances
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    print("Read %s sentence pairs" % len(data))
    pairs = clean_pairs(data)
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    #Make word lists from sentences in pairs
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Input Language\tTrimmed Vocabulary Size")
    print(input_lang.name, input_lang.n_words)
    print("Output Language\tTrimmed Vocabulary Size")
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs, test

In [22]:
input_lang, output_lang, tr_pairs, pairst = prepareData('hindi', 'english')
print(random.choice(tr_pairs))

Read 101322 sentence pairs
Trimmed to 100926 sentence pairs
Input Language	Trimmed Vocabulary Size
hindi 43447
Output Language	Trimmed Vocabulary Size
english 29399
['अब क्या करें' 'what do we do']


In [23]:
print(random.choice(tr_pairs))
from sklearn.model_selection import train_test_split
 
#split data into train and test set
#pairs, val_pairs = train_test_split(tr_pairs, test_size=0.1, random_state = 12)

['और जीवित ग्रंथों के अनुवाद का कहना है मैं इसका इलाज नहीं करूँगा इसका मै इलाज नहीं कर सकता'
 'and the translations of the surviving texts say this i will not treat this i can not treat']


In [24]:
print(len(tr_pairs))
print(len(pairst))

100926
24102


In [25]:
#vectorize sentences - using unique index for the word rather than entire one hot vector for each word
def indexesFromSentence(lang, sentence):
    indexes = []
    for word in sentence.split(' '):
        try:
            indexes.append(lang.word2index[word])
        except:
            indexes.append(lang.word2index["UNK"])
    return indexes

#one hot vector representation
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = torch.LongTensor(indexes).view(-1).cuda()
    return result

#convert pair of sentence to pair of tensors
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
#make batches of data
def batchify(data, batch_size):
    shuffle(data)

    num_of_batches = len(data) // batch_size
    batches = list(range(num_of_batches))

    for batch_num in range(num_of_batches):
        input_tensor = list(range(batch_size))
        target_tensor = list(range(batch_size))
        index = 0  

        for pair in range((batch_num*batch_size),((batch_num+1)*batch_size)):
            input_tensor[index], target_tensor[index] = tensorsFromPair(data[pair])
            index += 1

        batches[batch_num] = (input_tensor, target_tensor)

    return batches
 
#append EOS tag to end of shorter sentences so that all in batch are same length
def pad_batch(batch):
    padded_inputs = torch.nn.utils.rnn.pad_sequence(batch[0],padding_value=EOS_token)
    padded_targets = torch.nn.utils.rnn.pad_sequence(batch[1],padding_value=EOS_token)
    return (padded_inputs, padded_targets)

In [26]:
class EncoderRNN(nn.Module):
    def __init__(self,input_size,hidden_size,layers=1,dropout_p=0.1, bidirectional=True):
        super(EncoderRNN, self).__init__()
        #Model parameters
        self.directions = 2 if bidirectional else 1
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = layers
        self.dropout_p = dropout_p if self.n_layers > 1 else 0
        self.bidirectional = bidirectional
        
        #Construct the layers
        self.embed = nn.Embedding(self.input_size,self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(input_size=self.hidden_size,hidden_size=self.hidden_size,
                            num_layers=self.n_layers,dropout=self.dropout_p,bidirectional=self.bidirectional,
                            batch_first=False)
        self.fc = nn.Linear(self.hidden_size*self.directions, self.hidden_size)

    def forward(self, input, h_hidden, c_hidden):
        embedded = self.embed(input)
        embedded = self.dropout(embedded)
        #embedded tensor into RNN layer - (batchsize, maxlength,2*hidden)
        hiddens, outputs = self.lstm(embedded, (h_hidden, c_hidden))
        return hiddens, outputs

    def initHidden(self, batch_size):
        #for first time step, hidden state is empty vector
        h_hidden = Variable(torch.zeros(self.n_layers*self.directions,batch_size, self.hidden_size, device=device))
        c_hidden = Variable(torch.zeros(self.n_layers*self.directions,batch_size, self.hidden_size, device=device))
        return h_hidden, c_hidden

In [27]:
class DecoderAttn(nn.Module):
    def __init__(self, hidden_size, output_size, layers=1, dropout_p=0.1, bidirectional=True):
        super(DecoderAttn, self).__init__()
        #Model parameters
        self.directions = 2 if bidirectional else 1
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.n_layers = layers
        self.dropout_p = dropout_p if self.n_layers > 1 else 0 
        self.bidirectional = bidirectional
        
        #construct layers
        self.embed = nn.Embedding(self.output_size,self.hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        #Construct layer that calculates attentional weights
        self.score_learner = nn.Linear(self.hidden_size*self.directions, self.hidden_size*self.directions)
        self.lstm = nn.LSTM(input_size=self.hidden_size,hidden_size=self.hidden_size,
                            num_layers=self.n_layers,dropout=self.dropout_p, bidirectional=self.bidirectional,
                            batch_first=False)
        #Construct layer that compresses the combined matrix of the input embeddings and the encoder inputs after attention has been applied
        self.context_combiner = nn.Linear((hidden_size*self.directions)+(hidden_size*self.directions), hidden_size)
        self.tanh = nn.Tanh()
        #Output fc layer - to determine weights
        #1 weight per word in seq so dim = hiddensize*outputvocab
        self.output = nn.Linear(self.hidden_size, self.output_size)
        #weights should sum to 1
        self.soft = nn.Softmax(dim=1)
        self.log_soft = nn.LogSoftmax(dim=1)


    def forward(self, input_data, h_hidden, c_hidden, encoder_hiddens):
        #Input word indices, should have dim(1, batch_size), output will be (1, batch_size, embedding_dim)
        embedded = self.embed(input_data)
        embedded = self.dropout(embedded)
        batch_size = embedded.shape[1]
        hiddens, outputs = self.lstm(embedded, (h_hidden, c_hidden))
        #To determine weights,concatenation of the previous hidden state of the Decoder & embedded decoder output from prev step
        top_hidden = outputs[0].view(self.n_layers,self.directions,hiddens.shape[1],self.hidden_size)[self.n_layers-1]
        top_hidden = top_hidden.permute(1,2,0).contiguous().view(batch_size,-1, 1)
        #dim = (batchsize, hidden+embedding size)
        prep_scores = self.score_learner(encoder_hiddens.permute(1,0,2))
        #compute attention scores (weights) - global attention
        scores = torch.bmm(prep_scores, top_hidden)
        #get attention distribution; dim = (batchsize,seqlength)
        attn_scores = self.soft(scores)
        #weighted sum of encoder states - apply attention weights; Remove extra dimension, dim = (batch_size, 2*encoder_hidden_size)
        con_mat = torch.bmm(encoder_hiddens.permute(1,2,0),attn_scores)
        #fc+tanh
        h_tilde = self.tanh(self.context_combiner(torch.cat((con_mat,top_hidden),dim=1).view(batch_size,-1)))
        #pred for next word in seq
        pred = self.output(h_tilde)
        #A dense softmax layer to generate prob dist. over the target vocabulary
        pred = self.log_soft(pred)

        return pred, outputs

In [ ]:
clip = 5.0
teacher_forcing_ratio = 0.5

In [ ]:
#training batch-wise
def train(train_batches, encoder, decoder, encoder_optimizer, decoder_optimizer,criterion):
    losses = 0
    #completing forward pass for each sentence in batch
    for batch in train_batches:
        (input_batch, target_batch) = pad_batch(batch)
        #print(input_batch.shape)
        #clear the gradients in the optimizers
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        loss = 0
        #Initialize encoder hidden state
        enc_h_hidden, enc_c_hidden = encoder.initHidden(input_batch.shape[1])
        #meaning of whole sentence in hidden vector - context vector
        #run forward pass through encoder on entire sequence - in parallel for each sentence in batch
        enc_hiddens, enc_outputs = encoder(input_batch, enc_h_hidden, enc_c_hidden)

        #give SOS as input for first time step
        decoder_input = Variable(torch.LongTensor(1,input_batch.shape[1]).fill_(output_lang.word2index.get("SOS")).cuda())
        #Below tensors will hold the states of the previous time step
        dec_h_hidden = enc_outputs[0]
        dec_c_hidden = enc_outputs[1]

        #using the "Teacher forcing" - passing the actual output sentence as the input to decoder
#         if(random.random() < teacher_forcing_ratio):
#             use_teacher_forcing = True
#         else:
#             use_teacher_forcing = False
        
        use_teacher_forcing = True
        
        #Step through target sequence
        if use_teacher_forcing:
            for i in range(target_batch.shape[0]):
                #Forward pass through decoder - encoder outputs and predicted word index from previous time
                pred, dec_outputs = decoder(decoder_input, dec_h_hidden, dec_c_hidden, enc_hiddens)
                #Feed target as input to next item in the sequence
                decoder_input = target_batch[i].view(1,-1)
                dec_h_hidden = dec_outputs[0]
                dec_c_hidden = dec_outputs[1]

                loss += criterion(pred,target_batch[i])
        else:
            for i in range(target_batch.shape[0]):
                pred, dec_outputs = decoder(decoder_input, dec_h_hidden, dec_c_hidden, enc_hiddens)
                #word with max prob becomes predicted word
                topv, topi = pred.topk(1,dim=1)
                ni = topi.view(1,-1)
                #using predicted word as input to next time-step
                decoder_input = ni
                dec_h_hidden = dec_outputs[0]
                dec_c_hidden = dec_outputs[1]
                #computing accuracy of predicted translation to actual translation. summing over the loss for each word
                loss += criterion(pred,target_batch[i])        

        #backpropagate the loss
        loss.backward()
        #Clip the gradients
        torch.nn.utils.clip_grad_norm_(encoder.parameters(),clip)
        torch.nn.utils.clip_grad_norm_(decoder.parameters(),clip)
        #updated weights slightly enhance accuracy
        encoder_optimizer.step()
        decoder_optimizer.step()
        #summing over loss of several sentences in batch
        batch_loss = loss.item() / target_batch.shape[0]
        losses += batch_loss

    return losses / len(train_batches)

In [ ]:
def test(test_batches, encoder, decoder, criterion):
    with torch.no_grad():
        losses = 0
        for batch in test_batches:
            (input_batch, target_batch) = pad_batch(batch)

            loss = 0
            enc_h_hidden, enc_c_hidden = encoder.initHidden(input_batch.shape[1])
            enc_hiddens, enc_outputs = encoder(input_batch, enc_h_hidden, enc_c_hidden)

            decoder_input = Variable(torch.LongTensor(1,input_batch.shape[1]).fill_(output_lang.word2index.get("SOS")).cuda())
            dec_h_hidden = enc_outputs[0]
            dec_c_hidden = enc_outputs[1]

            for i in range(target_batch.shape[0]):
                pred, dec_outputs = decoder(decoder_input, dec_h_hidden, dec_c_hidden, enc_hiddens)
                topv, topi = pred.topk(1,dim=1)
                ni = topi.view(1,-1)
                #no teacher forcing during testing,keeping the predicted word as input to next step
                decoder_input = ni
                dec_h_hidden = dec_outputs[0]
                dec_c_hidden = dec_outputs[1]

                loss +=criterion(pred,target_batch[i])

            batch_loss = loss.item() / target_batch.shape[0]
            losses += batch_loss
            
    return losses / len(test_batches)

In [28]:
#return translation of a sentence
def evaluate(encoder, decoder, sentence, cutoff_length):
    with torch.no_grad():
        # Encode the input as state vectors.
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_tensor = input_tensor.view(-1,1)
        enc_h_hidden, enc_c_hidden = encoder.initHidden(1)
        enc_hiddens, enc_outputs = encoder(input_tensor, enc_h_hidden, enc_c_hidden)
        # Generate empty target sequence of length 1, Populate the first character of target sequence with the start character
        decoder_input = Variable(torch.LongTensor(1,1).fill_(output_lang.word2index.get("SOS")).cuda())
        dec_h_hidden = enc_outputs[0]
        dec_c_hidden = enc_outputs[1]

        decoded_words = []

        for di in range(cutoff_length):
            pred, dec_outputs = decoder(decoder_input, dec_h_hidden, dec_c_hidden, enc_hiddens)
            #Sample a token
            topv, topi = pred.topk(1,dim=1)
            ni = topi.item()
            #find stop character
            if(ni == output_lang.word2index.get("EOS")):
                decoded_words.append("EOS")
                break
            else:
                decoded_words.append(output_lang.index2word[ni])
            # Update the target sequence (of length 1)
            decoder_input = Variable(torch.LongTensor(1,1).fill_(ni).cuda())
            dec_h_hidden = dec_outputs[0]
            dec_c_hidden = dec_outputs[1]

        output_sentence = ' '.join(decoded_words[:-1])
        return output_sentence

In [29]:
import time
import math
from random import shuffle
from torch.autograd import Variable

In [ ]:
def evaluate_randomly(encoder, decoder, pairs, n=1):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_sentence = evaluate(encoder, decoder, pair[0],cutoff_length=150)
        print('<', output_sentence)
        print('')

In [ ]:
def timeSpent(s):
    m = math.floor(s / 60)
    h = math.floor(m / 60)
    s -= m * 60
    m -= h * 60
    return '%dh %dm %ds' % (h, m, s)

In [ ]:
output_file = "save3_weight512"

In [ ]:
def train_test(epochs, test_every, learning_rate, train_pairs, test_pairs, input_lang, output_lang, batch_size, test_batch_size, encoder, decoder,criterion):
    #test_batches = batchify(test_pairs, test_batch_size)
    #keep track of losses
    train_plot_losses = []
    val_plot_losses = []
    start = time.time()
    #Cycle through epochs
    for i in range(1,epochs+1):
        encoder.train()
        decoder.train()
        #Initialize Optimizer
        encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
        decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

        batches = batchify(train_pairs, batch_size)
        train_loss = train(batches, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        train_plot_losses.append(train_loss)

        now = time.time()
        print("Iteration: %s \nLearning Rate: %s \nTime: %s \nTrain Loss: %s \n" % (i, learning_rate, timeSpent(now-start), train_loss))

#         if(i % test_every == 0):
#             test_loss = test(test_batches, encoder, decoder, criterion)
#             val_plot_losses.append(test_loss)
#             print("Test Loss: %s" % (test_loss))
#             evaluate_randomly(encoder, decoder, test_pairs,1)

        torch.save(encoder.state_dict(), output_file+'_enc.pt')
        torch.save(decoder.state_dict(), output_file+'_dec.pt')

        history = {'train_loss' : train_plot_losses,
               'val_loss': val_plot_losses,
           }

    return history

In [30]:
hidden_size = 512
encoder = EncoderRNN(input_lang.n_words, hidden_size, layers=3,dropout_p=0.1, bidirectional=True).to(device)
decoder = DecoderAttn(hidden_size, output_lang.n_words, layers=3,dropout_p=0.1, bidirectional=True).to(device)

In [31]:
encoder

EncoderRNN(
  (embed): Embedding(43447, 512)
  (dropout): Dropout(p=0.1, inplace=False)
  (lstm): LSTM(512, 512, num_layers=3, dropout=0.1, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=512, bias=True)
)

In [ ]:
# Specify loss function, ignore the padded "SOS" index so it does not contribute to loss.
#criterion = nn.NLLLoss(ignore_index=0)
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [ ]:
val_pairs=[]
history = train_test(epochs=12, test_every= 1,learning_rate= 0.001, train_pairs = tr_pairs, test_pairs = val_pairs, input_lang=input_lang, output_lang = output_lang, batch_size = 64, 
               test_batch_size = 64, encoder = encoder, decoder = decoder, criterion = criterion)

In [32]:
pred = []
def evaluate_Randomly(encoder, decoder, pairs1):
  for i in range(len(pairs1)):
    pair = pairs1[i]
    output_sentence = evaluate(encoder, decoder,pair[0],cutoff_length=150)
    output_sentence = output_sentence+"\n"
    pred.append(output_sentence)
    f1.writelines(output_sentence)

In [33]:
f1= open("answer.txt",'w')

In [36]:
evaluate_Randomly(encoder, decoder, pairst)

In [37]:
f1.close()

In [34]:
#load the state from dictionary 
encoder.load_state_dict(torch.load("/kaggle/input/lstm1024/save3_weight512_enc.pt"))
decoder.load_state_dict(torch.load("/kaggle/input/lstm1024/save3_weight512_dec.pt"))

<All keys matched successfully>

In [35]:
#set to inference mode
encoder.eval().to(device)
decoder.eval().to(device)

DecoderAttn(
  (embed): Embedding(29399, 512)
  (dropout): Dropout(p=0.1, inplace=False)
  (score_learner): Linear(in_features=1024, out_features=1024, bias=True)
  (lstm): LSTM(512, 512, num_layers=3, dropout=0.1, bidirectional=True)
  (context_combiner): Linear(in_features=2048, out_features=512, bias=True)
  (tanh): Tanh()
  (output): Linear(in_features=512, out_features=29399, bias=True)
  (soft): Softmax(dim=1)
  (log_soft): LogSoftmax(dim=1)
)

In [ ]:
#len(pred)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

In [ ]:
%matplotlib inline

#PLot the train and validation loss 
plt.figure()
fig, ax = plt.subplots()
plt.plot(history['train_loss'], label = 'train loss')
plt.plot(history['val_loss'], label = 'val loss')
plt.title('Loss vs Training steps')
plt.xlabel('Training epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()